In [134]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [135]:
class MyDataset(Dataset):
    # 构造函数
    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
    # 返回数据集大小
    def __len__(self):
        return self.data_tensor.size(0)
    # 返回索引的数据与标签
    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index]

In [144]:
def map_array(arr):
    mapped_array = []
    for num in arr:
        if num == 0:
            mapped_array.append([1, 0])
        elif num == 1:
            mapped_array.append([0, 1])
    return mapped_array

In [152]:
from scipy.io import loadmat
file6 = "D:\\320 Deep-learning\data\\1118_S6_time1000.mat"
data6 = loadmat(file6)
file12 = "D:\\320 Deep-learning\data\\1111_S12_time1000.mat"
data12 = loadmat(file12)
file24 = "D:\\320 Deep-learning\data\\1111_S24_time1000.mat"
data24 = loadmat(file24)
""" experiment
X_train = data['Input']
X_train = torch.from_numpy(X_train).float()
y_train = data['Stability']
y_train = y_train.astype(np.float32) / 255
y_train = torch.tensor(y_train)
len_train = len(y_train)
X_test = data['PreInput']
X_test = torch.from_numpy(X_test).float()
y_test = data['PreSta']
y_test = y_test.astype(np.float32) / 255
y_test = torch.tensor(y_test)
len_test = len(y_test) """

#simulation
X6 = data6['Input_Abundance']
X6 = torch.from_numpy(X6).float()
#y6 = data6['Std_total']
#y6 = -np.log10(y6)
#y6 = y6.T
y6 = data6['Stability']
y66 = map_array(y6)
print(map_array(y6))
y6 = torch.from_numpy(y6).float()

X12 = data12['Input_RelativeAbundance']
X12 = torch.from_numpy(X12).float()
y12 = data12['Stability']
y12 = torch.from_numpy(y12).float()

X24 = data24['Input_RelativeAbundance']
X24 = torch.from_numpy(X24).float()
y24= data24['Stability']
y24 = torch.from_numpy(y24).float()

t = 700
X_train = X6[0:t,:]
y_train = y66[0:t]
X_test = X6[t:1000,:]
y_test = y66[t:1000]

len_train = len(y_train)
len_test = len(y_test)

# 将数据封装成Dataset
train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0],

In [148]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 4

""" 需不需要随机采样？ 随机采样和shuffle是不是一样的 """
""" # define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_index)
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, 
                                           sampler = train_sampler, num_workers = num_workers) """

# prepare data loaders
train_dataloader = DataLoader(dataset = train_dataset, # 传入的数据集, 必须参数
                               batch_size = batch_size,       # 输出的batch大小
                               shuffle = True,       # 数据是否打乱
                               num_workers = num_workers)      # 进程数, 0表示只有主进程
test_dataloader = DataLoader(dataset = test_dataset, # 传入的数据集, 必须参数
                               batch_size = batch_size,       # 输出的batch大小
                               shuffle = True,       # 数据是否打乱
                               num_workers = num_workers)      # 进程数, 0表示只有主进程

#for data,target in train_dataloader:
#    print(target.size(1))

In [149]:
import torch.nn as nn
import torch.nn.functional as F

# define NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(54, 1)
    def forward(self,x):
        x = self.fc1(x)
        #x = F.sigmoid(self.fc1(x))
        return x

# initialize the NN
model = Net()
#print(model)

In [150]:
# specify loss function
criterion = nn.BCEWithLogitsLoss()
#criterion = nn.CrossEntropyLoss()
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [151]:
# number of epochs to train the model
n_epochs = 100
train_loss_min = np.Inf  # set initial "min" to infinity

for epoch in range(n_epochs):
    train_loss = 0
    # train the model #
    loss = 0
    for data, target in train_dataloader: 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
    # calculate average loss over an epoch
    train_loss = train_loss / len_train
    
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    
    # save model
    if train_loss <= train_loss_min:
        print('Train loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        train_loss_min,
        train_loss))
        torch.save(model.state_dict(), 'model.pt')
        train_loss_min = train_loss

AttributeError: 'list' object has no attribute 'size'

In [141]:

model.load_state_dict(torch.load('model.pt'))
#for name, parameters in model.named_parameters():
#    print(name, ':', parameters.size())
print(model.fc1.weight)

Parameter containing:
tensor([[-1.0976,  0.1982, -0.1560,  0.2119, -0.0891, -0.2251,  0.2608, -0.3910,
          0.2025, -1.4738, -2.7629, -0.5045, -0.5454,  0.3039, -0.4025, -1.7458,
         -0.3177, -1.3588,  0.2854, -0.1288, -0.1923, -0.4056,  0.1669, -0.2874,
         -0.6805,  0.3351, -1.4278,  0.2097,  0.1359, -0.3311, -1.2077, -0.1384,
         -1.0069,  0.4476,  0.4758, -0.3405,  0.1287, -0.1422,  0.1944, -0.8249,
          0.8655, -0.6114,  0.1107, -0.6839, -0.7287, -0.7014, -0.7409,  0.3331,
          0.4360,  0.7154,  0.0772, -0.5540, -0.5082,  0.0453]],
       requires_grad=True)


In [142]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
correct = 0
OUTPUT = []
S_pre = torch.Tensor(len_test,1)
S_true = torch.Tensor(len_test,1)

for i in range(len_test):
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(X_test[i])
    o = output.tolist()
    OUTPUT.append(o)
    # calculate the loss
    loss = criterion(output, y_test[i])
    # update test loss 
    test_loss += loss.item()
    # use the output to predict y
    if(output<0.01): S_pre[i] = 0
    else: S_pre[i] = 1
    if(y_test[i]<0.01): S_true[i] = 0
    else: S_true[i] = 1   
    correct += S_pre[i].eq(S_true[i])

# calculate and print avg test loss
test_loss = test_loss/len_test
correct_ratio = correct/len_test
print('Test Loss: {:.6f}\n'.format(test_loss))
print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
    100*correct_ratio,correct,len_test))


Test Loss: 0.343021


Test Accuracy: 90% (271/300)


In [143]:
test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

for data, target in test_dataloader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    #output.argmax(1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(len(target)):
        if (target.data[i]>0.01):
            label = 1
        else:
            label = 0
        #label = torch.tensor(target.data[i], dtype=torch.int64)
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
""" test_loss = test_loss/len(test_dataloader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss)) """

for i in range(2):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of     0: 100% (271/271)
Test Accuracy of     1:  0% ( 0/29)

Test Accuracy (Overall): 90% (271/300)
